In [1]:
import pandas as pd
import re

In [2]:
# Bringing in the ten csvs created during data gathering, and combining these csvs into a single csv.

cs_1 = pd.read_csv('../data/Caissabase1.csv')
cs_2 = pd.read_csv('../data/Caissabase2.csv')
cs_3 = pd.read_csv('../data/Caissabase3.csv')
cs_4 = pd.read_csv('../data/Caissabase4.csv')
cs_5 = pd.read_csv('../data/Caissabase5.csv')
cs_6 = pd.read_csv('../data/Caissabase6.csv')
cs_7 = pd.read_csv('../data/Caissabase7.csv')
cs_8 = pd.read_csv('../data/Caissabase8.csv')
cs_9 = pd.read_csv('../data/Caissabase9.csv')
cs_10 = pd.read_csv('../data/Caissabase10.csv')

caissabase = pd.concat([cs_1,cs_2,cs_3,cs_4,cs_5,cs_6,cs_7,cs_8,cs_9,cs_10])

In [3]:
# Cleaning up initial dataframe.

caissabase = caissabase.drop(caissabase.columns[0], axis = 1)
caissabase.columns = ['games']

In [4]:
caissabase['event'] = caissabase['games'].str.extract('(\[Event[^\]]*\])')
caissabase['event'] = caissabase['event'].str.extract('"([^#]*)"')

In [5]:
caissabase['white_elo'] = caissabase['games'].str.extract('(\[WhiteElo[^\]]*\])')
caissabase['white_elo'] = caissabase['white_elo'].str.extract('"([^#]*)"')

In [6]:
caissabase['black_elo'] = caissabase['games'].str.extract('(\[BlackElo[^\]]*\])')
caissabase['black_elo'] = caissabase['black_elo'].str.extract('"([^#]*)"')

In [7]:
caissabase = caissabase.dropna(axis = 0, how = 'any', subset = ['white_elo','black_elo'])

In [8]:
caissabase = caissabase.astype({'white_elo':'int', 'black_elo':'int'})

In [9]:
caissabase = caissabase[caissabase['white_elo'].between(2000, 2890, 'both')]
caissabase = caissabase[caissabase['black_elo'].between(2000, 2890, 'both')]

In [10]:
caissabase = caissabase[~caissabase['event'].str.contains('rapid| rpd |blitz|bltz|bullet|exhibition| exh |titled arena|titled tuesday|titled tues|speed|simul|960|random', case = False, na = False)]

In [11]:
caissabase['result'] = caissabase['games'].str.extract('(\[Result[^\]]*\])')
caissabase['result'] = caissabase['result'].str.extract('"([^#]*)"')

In [12]:
caissabase['site'] = caissabase['games'].str.extract('(\[Site[^\]]*\])')
caissabase['site'] = caissabase['site'].str.extract('"([^#]*)"')

In [13]:
caissabase['date'] = caissabase['games'].str.extract('(\[Date[^\]]*\])')
caissabase['date'] = caissabase['date'].str.extract('"([^#]*)"')

In [14]:
caissabase['white'] = caissabase['games'].str.extract('(\[White[^\]]*\])')
caissabase['white'] = caissabase['white'].str.extract('"([^#]*)"')

In [15]:
caissabase['white_fide_id'] = caissabase['games'].str.extract('(\[WhiteFideId[^\]]*\])')
caissabase['white_fide_id'] = caissabase['white_fide_id'].str.extract('"([^#]*)"')

In [16]:
caissabase['white_title'] = caissabase['games'].str.extract('(\[WhiteTitle[^\]]*\])')
caissabase['white_title'] = caissabase['white_title'].str.extract('"([^#]*)"')

In [17]:
caissabase['black'] = caissabase['games'].str.extract('(\[Black[^\]]*\])')
caissabase['black'] = caissabase['black'].str.extract('"([^#]*)"')

In [18]:
caissabase['black_fide_id'] = caissabase['games'].str.extract('(\[BlackFideId[^\]]*\])')
caissabase['black_fide_id'] = caissabase['black_fide_id'].str.extract('"([^#]*)"')

In [19]:
caissabase['black_title'] = caissabase['games'].str.extract('(\[BlackTitle[^\]]*\])')
caissabase['black_title'] = caissabase['black_title'].str.extract('"([^#]*)"')

In [20]:
caissabase['eco'] = caissabase['games'].str.extract('(\[ECO[^\]]*\])')
caissabase['eco'] = caissabase['eco'].str.extract('"([^#]*)"')

In [21]:
caissabase['event_date'] = caissabase['games'].str.extract('(\[EventDate[^\]]*\])')
caissabase['event_date'] = caissabase['event_date'].str.extract('"([^#]*)"')

In [22]:
caissabase['opening'] = caissabase['games'].str.extract('(\[Opening[^\]]*\])')
caissabase['opening'] = caissabase['opening'].str.extract('"([^#]*)"')

In [23]:
caissabase['variation'] = caissabase['games'].str.extract('(\[Variation[^\]]*\])')
caissabase['variation'] = caissabase['variation'].str.extract('"([^#]*)"')

In [24]:
caissabase['moves'] = caissabase['games'].str.extract('(\\n\\n[^%]*)')

In [25]:
caissabase = caissabase.drop(['games'], axis = 1)

In [26]:
caissabase['moves'] = caissabase['moves'].str.replace(r'1-0', r'')
caissabase['moves'] = caissabase['moves'].str.replace(r'0-1', r'')
caissabase['moves'] = caissabase['moves'].str.replace(r'1/2-1/2', r'')
caissabase['@'] = '@'
caissabase['moves'] = caissabase['moves'] + caissabase['@']

In [27]:
caissabase = caissabase.drop(['@'], axis = 1)

In [28]:
for x in range (1,21):
    y = ' ' + str(x + 1)
    x = str(x)
    caissabase[x] = caissabase['moves'].str.extract('('+x+'\.[^@.]*)')
    caissabase[x] = caissabase[x].str.replace(y, '')
    caissabase[x] = caissabase[x].str.replace(x + '\.', '')

<ipython-input-28-336d59ff31a9>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  caissabase[x] = caissabase[x].str.replace(x + '\.', '')


In [29]:
caissabase = caissabase.drop(['moves'], axis = 1)

In [30]:
for x in range (1,21):
    x = str(x)
    w = x + 'w'
    b = x + 'b'
    caissabase[x] = caissabase[x].str.strip()
    caissabase[[w, b]] = caissabase[x].str.split(' ', n = 1, expand = True)

In [31]:
caissabase = caissabase.drop(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'], axis = 1)

In [32]:
caissabase['date'] = pd.to_datetime(caissabase['date'], errors = 'coerce')

In [33]:
caissabase = caissabase.loc[caissabase['date']>='2000-01-01']

In [34]:
caissabase = caissabase.dropna(axis = 0, how = 'any', subset = ['1w','1b','2w','2b','3w','3b','4w','4b'])

In [35]:
caissabase = caissabase[caissabase['1w'].isin(['Bh3','Nd6+','f5','Nf5']) == False]

In [36]:
caissabase = caissabase[caissabase['1b'].isin(['{ Note this is not a standard game of chess as castling was not\nallowed } 1','f62']) == False]

In [37]:
caissabase = caissabase.loc[caissabase['result'] != '*']

In [38]:
caissabase = caissabase.reset_index(drop =True)

In [39]:
caissabase.to_csv('../data/caissabase_df.csv')